<a href="https://colab.research.google.com/github/gulabpatel/InstructModels/blob/main/Mistral/Mistral7llamaindex_MedicalChatBot_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code walkthrogh video: https://www.youtube.com/watch?v=1mH1BvBJCl0

Relevant Links:

https://huggingface.co/thenlper/gte-large

https://python.langchain.com/docs/integrations/llms/llamacpp

In [1]:
!pip install -q pypdf
!pip install -q python-dotenv
!pip install -q transformers

In [2]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install  llama-cpp-python --no-cache-dir

In [3]:
!pip install -q llama-index

In [4]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

In [5]:
documents = SimpleDirectoryReader("/content/").load_data()

In [ ]:
documents

In [7]:
import torch
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url = 'https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf',
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path = None,
    temperature = 0.1,
    max_new_tokens = 256,
    # mistral has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window = 3900,
    # kwargs to pass to __call__()
    generate_kwargs = {},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs = {"n_gpu_layers": -1},
    # transform inputs into Llama2 format
    messages_to_prompt = messages_to_prompt,
    completion_to_prompt = completion_to_prompt,
    verbose = True,
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [8]:
!pip -q install sentence-transformers

In [9]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding

embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="thenlper/gte-large")
)

In [10]:
service_context = ServiceContext.from_defaults(
    chunk_size=256,
    llm=llm,
    embed_model=embed_model
)

In [11]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [12]:
query_engine = index.as_query_engine()
response = query_engine.query("What is Fibromyalgia?")

In [13]:
print(response)

 Fibromyalgia is a long-term condition that causes pain and tenderness all over your body. It's thought to be caused by your nervous system in your brain and spine not being able to control or process pain signals from other parts of your body. The condition is also linked to poor sleep, feeling you can't concentrate or remember things, extreme and constant tiredness that doesn't improve with sleep or rest (known as fatigue), and many other symptoms that affect different parts of your body.


In [ ]:
while True:
  query=input()
  response = query_engine.query(query)
  print(response)

Llama.generate: prefix-match hit


 The symptoms of Fibromyalgia include pain that spreads throughout your entire body, with certain parts such as your neck and back feeling particularly painful, tiredness, fatigue, difficulty sleeping, and aching and stiffness.
